In [1]:
# =============================================================
# ADVANCED MULTI-SPRINKLER HYBRID SYSTEM (NO GRAPH VERSION)
# BDI + DELIBERATIVE + MODEL-BASED REFLEX + LEARNING
# =============================================================

import random
import numpy as np

# -----------------------------
# PARAMETERS
# -----------------------------
SPRINKLERS = ["Sprklr1", "Sprklr2", "Sprklr3"]
TARGET = 65
EPISODES = 100
ALPHA = 0.1
GAMMA = 0.9
EPSILON = 0.2

PRESSURE_MODES = ["OFF", "LOW", "HIGH"]
CYCLE_TIMES = [20, 24, 4, 8]   # 8PM, 12AM, 4AM, 8AM
MAX_CYCLES = 4

Q = {}

# -----------------------------
# WEATHER MODEL (Base values)
# -----------------------------
def base_weather(weather):
    if weather == "hot":
        return 35, 30
    elif weather == "humid":
        return 28, 80
    elif weather == "rain":
        return 24, 90
    elif weather == "snow":
        return 2, 70
    else:
        return 30, 50

# -----------------------------
# TEMPERATURE VARIATION MODEL
# 8PM → 1AM increase
# 1AM → 9PM decrease
# -----------------------------
def dynamic_temperature(base_temp, hour):
    if hour <= 1 or hour >= 20:
        return base_temp + 2
    elif 1 < hour <= 13:
        return base_temp + (13 - hour)
    else:
        return base_temp

def evaporation_rate(temp, humidity):
    return max(1, (temp / 6) - (humidity / 100))

# -----------------------------
# STATE
# -----------------------------
def get_state(moisture):
    return int(moisture // 5) * 5

# -----------------------------
# STEP
# -----------------------------
def step(moisture, weather, pressure, hour):

    base_temp, humidity = base_weather(weather)
    temp = dynamic_temperature(base_temp, hour)
    evap = evaporation_rate(temp, humidity)

    if weather in ["rain", "snow"]:
        pressure = "OFF"

    if pressure == "OFF":
        water = 0
        cost = 0
    elif pressure == "LOW":
        water = 8
        cost = 1
    else:
        water = 15
        cost = 3

    new_moisture = moisture + water - evap
    new_moisture = max(0, min(100, new_moisture))

    reward = -abs(TARGET - new_moisture) - cost
    return new_moisture, reward, temp, evap, pressure

# =============================================================
# TRAINING PHASE
# =============================================================
weather_states = ["hot", "humid", "rain", "snow"]

for ep in range(EPISODES):

    weather = random.choice(weather_states)

    for s in SPRINKLERS:

        moisture = random.randint(20, 40)

        for _ in range(3):

            state = get_state(moisture)

            if random.random() < EPSILON:
                action = random.choice(PRESSURE_MODES)
            else:
                action = max(PRESSURE_MODES,
                             key=lambda a: Q.get((s, state, a), 0))

            new_m, reward, _, _, action = step(moisture, weather, action, 20)
            next_state = get_state(new_m)

            old_q = Q.get((s, state, action), 0)
            future_q = max([Q.get((s, next_state, a), 0)
                            for a in PRESSURE_MODES])

            Q[(s, state, action)] = old_q + ALPHA * (
                reward + GAMMA * future_q - old_q
            )

            moisture = new_m

print("\n================ TRAINING COMPLETE =================")
print("Episodes:", EPISODES)

print("\nLearned Policy Snapshot:")
for s in SPRINKLERS:
    print(f"\n{s} learned best actions by moisture state:")
    for m in [20,30,40,50,60]:
        best = max(PRESSURE_MODES,
                   key=lambda a: Q.get((s,m,a),0))
        print(f"  Moisture {m} → {best}")

print("\nSystem ready for deployment.\n")

# =============================================================
# USER INPUT PHASE
# =============================================================
moisture_levels = {s:20 for s in SPRINKLERS}

for day in range(5):

    weather = input(f"\nEnter weather for Day {day+1} (hot/humid/rain/snow): ").lower()

    print("\n===================================================")
    print(f"DAY {day+1} | Weather: {weather}")
    print("Initial Moisture:", moisture_levels)

    beliefs = {"weather":weather, "moisture":moisture_levels.copy()}
    desires = {"uniform_target":TARGET}

    if weather in ["rain","snow"]:
        intentions = ["system_off"]
    else:
        intentions = ["water_until_target"]

    print("\nBDI LAYER")
    print("Beliefs:", beliefs)
    print("Desires:", desires)
    print("Intentions:", intentions)

    if "system_off" in intentions:
        print("System OFF due to weather.")
        continue

    print("\nDELIBERATIVE PLANNING")
    print("Plan: Water zones in order of lowest moisture to reduce variance.")

    for cycle in range(MAX_CYCLES):

        hour = CYCLE_TIMES[cycle]
        print(f"\n--- Cycle {cycle+1} at {hour}:00 ---")

        sorted_sprinklers = sorted(SPRINKLERS,
                                   key=lambda s: moisture_levels[s])

        for s in sorted_sprinklers:

            if moisture_levels[s] >= TARGET:
                print(f"{s} reached target → Reflex OFF")
                continue

            state = get_state(moisture_levels[s])
            action = max(PRESSURE_MODES,
                         key=lambda a: Q.get((s,state,a),0))

            start_m = moisture_levels[s]

            new_m, _, temp, evap, action = step(start_m, weather, action, hour)

            print(f"{s}")
            print(f"  State: {state}")
            print(f"  Action: {action}")
            print(f"  Temp: {round(temp,2)}")
            print(f"  Evaporation: {round(evap,2)}")
            print(f"  Moisture Start: {round(start_m,2)}")
            print(f"  Moisture End: {round(new_m,2)}")

            moisture_levels[s] = new_m

    print("\nEnd of Day Moisture:", moisture_levels)

print("\n================ END OF 5 DAY SIMULATION ================")


================ TRAINING COMPLETE =================
Episodes: 100

Learned Policy Snapshot:

Sprklr1 learned best actions by moisture state:
  Moisture 20 → LOW
  Moisture 30 → LOW
  Moisture 40 → LOW
  Moisture 50 → HIGH
  Moisture 60 → OFF

Sprklr2 learned best actions by moisture state:
  Moisture 20 → HIGH
  Moisture 30 → HIGH
  Moisture 40 → OFF
  Moisture 50 → HIGH
  Moisture 60 → HIGH

Sprklr3 learned best actions by moisture state:
  Moisture 20 → LOW
  Moisture 30 → LOW
  Moisture 40 → HIGH
  Moisture 50 → HIGH
  Moisture 60 → HIGH

System ready for deployment.




Enter weather for Day 1 (hot/humid/rain/snow):  humid



DAY 1 | Weather: humid
Initial Moisture: {'Sprklr1': 20, 'Sprklr2': 20, 'Sprklr3': 20}

BDI LAYER
Beliefs: {'weather': 'humid', 'moisture': {'Sprklr1': 20, 'Sprklr2': 20, 'Sprklr3': 20}}
Desires: {'uniform_target': 65}
Intentions: ['water_until_target']

DELIBERATIVE PLANNING
Plan: Water zones in order of lowest moisture to reduce variance.

--- Cycle 1 at 20:00 ---
Sprklr1
  State: 20
  Action: LOW
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 20
  Moisture End: 23.8
Sprklr2
  State: 20
  Action: HIGH
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 20
  Moisture End: 30.8
Sprklr3
  State: 20
  Action: LOW
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 20
  Moisture End: 23.8

--- Cycle 2 at 24:00 ---
Sprklr1
  State: 20
  Action: LOW
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 23.8
  Moisture End: 27.6
Sprklr3
  State: 20
  Action: LOW
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 23.8
  Moisture End: 27.6
Sprklr2
  State: 30
  Action: HIGH
  Temp: 30
  Evaporation: 4.2
  Mo


Enter weather for Day 2 (hot/humid/rain/snow):  humid



DAY 2 | Weather: humid
Initial Moisture: {'Sprklr1': 33.53333333333333, 'Sprklr2': 46.533333333333324, 'Sprklr3': 33.53333333333333}

BDI LAYER
Beliefs: {'weather': 'humid', 'moisture': {'Sprklr1': 33.53333333333333, 'Sprklr2': 46.533333333333324, 'Sprklr3': 33.53333333333333}}
Desires: {'uniform_target': 65}
Intentions: ['water_until_target']

DELIBERATIVE PLANNING
Plan: Water zones in order of lowest moisture to reduce variance.

--- Cycle 1 at 20:00 ---
Sprklr1
  State: 30
  Action: LOW
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 33.53
  Moisture End: 37.33
Sprklr3
  State: 30
  Action: LOW
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 33.53
  Moisture End: 37.33
Sprklr2
  State: 45
  Action: HIGH
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 46.53
  Moisture End: 57.33

--- Cycle 2 at 24:00 ---
Sprklr1
  State: 35
  Action: HIGH
  Temp: 30
  Evaporation: 4.2
  Moisture Start: 37.33
  Moisture End: 48.13
Sprklr3
  State: 35
  Action: HIGH
  Temp: 30
  Evaporation: 4.2
  Moi

...


DAY 3 | Weather: hot
Initial Moisture: {'Sprklr1': 61.06666666666665, 'Sprklr2': 68.13333333333331, 'Sprklr3': 68.06666666666665}

BDI LAYER
Beliefs: {'weather': 'hot', 'moisture': {'Sprklr1': 61.06666666666665, 'Sprklr2': 68.13333333333331, 'Sprklr3': 68.06666666666665}}
Desires: {'uniform_target': 65}
Intentions: ['water_until_target']

DELIBERATIVE PLANNING
Plan: Water zones in order of lowest moisture to reduce variance.

--- Cycle 1 at 20:00 ---
Sprklr1
  State: 60
  Action: OFF
  Temp: 37
  Evaporation: 5.87
  Moisture Start: 61.07
  Moisture End: 55.2
Sprklr3 reached target → Reflex OFF
Sprklr2 reached target → Reflex OFF

--- Cycle 2 at 24:00 ---
Sprklr1
  State: 55
  Action: LOW
  Temp: 37
  Evaporation: 5.87
  Moisture Start: 55.2
  Moisture End: 57.33
Sprklr3 reached target → Reflex OFF
Sprklr2 reached target → Reflex OFF

--- Cycle 3 at 4:00 ---
Sprklr1
  State: 55
  Action: LOW
  Temp: 44
  Evaporation: 7.03
  Moisture Start: 57.33
  Moisture End: 58.3
Sprklr3 reached tar


Enter weather for Day 4 (hot/humid/rain/snow):  hot



DAY 4 | Weather: hot
Initial Moisture: {'Sprklr1': 59.933333333333316, 'Sprklr2': 68.13333333333331, 'Sprklr3': 68.06666666666665}

BDI LAYER
Beliefs: {'weather': 'hot', 'moisture': {'Sprklr1': 59.933333333333316, 'Sprklr2': 68.13333333333331, 'Sprklr3': 68.06666666666665}}
Desires: {'uniform_target': 65}
Intentions: ['water_until_target']

DELIBERATIVE PLANNING
Plan: Water zones in order of lowest moisture to reduce variance.

--- Cycle 1 at 20:00 ---
Sprklr1
  State: 55
  Action: LOW
  Temp: 37
  Evaporation: 5.87
  Moisture Start: 59.93
  Moisture End: 62.07
Sprklr3 reached target → Reflex OFF
Sprklr2 reached target → Reflex OFF

--- Cycle 2 at 24:00 ---
Sprklr1
  State: 60
  Action: OFF
  Temp: 37
  Evaporation: 5.87
  Moisture Start: 62.07
  Moisture End: 56.2
Sprklr3 reached target → Reflex OFF
Sprklr2 reached target → Reflex OFF

--- Cycle 3 at 4:00 ---
Sprklr1
  State: 55
  Action: LOW
  Temp: 44
  Evaporation: 7.03
  Moisture Start: 56.2
  Moisture End: 57.17
Sprklr3 reached 


Enter weather for Day 5 (hot/humid/rain/snow):  hot



DAY 5 | Weather: hot
Initial Moisture: {'Sprklr1': 58.799999999999976, 'Sprklr2': 68.13333333333331, 'Sprklr3': 68.06666666666665}

BDI LAYER
Beliefs: {'weather': 'hot', 'moisture': {'Sprklr1': 58.799999999999976, 'Sprklr2': 68.13333333333331, 'Sprklr3': 68.06666666666665}}
Desires: {'uniform_target': 65}
Intentions: ['water_until_target']

DELIBERATIVE PLANNING
Plan: Water zones in order of lowest moisture to reduce variance.

--- Cycle 1 at 20:00 ---
Sprklr1
  State: 55
  Action: LOW
  Temp: 37
  Evaporation: 5.87
  Moisture Start: 58.8
  Moisture End: 60.93
Sprklr3 reached target → Reflex OFF
Sprklr2 reached target → Reflex OFF

--- Cycle 2 at 24:00 ---
Sprklr1
  State: 60
  Action: OFF
  Temp: 37
  Evaporation: 5.87
  Moisture Start: 60.93
  Moisture End: 55.07
Sprklr3 reached target → Reflex OFF
Sprklr2 reached target → Reflex OFF

--- Cycle 3 at 4:00 ---
Sprklr1
  State: 55
  Action: LOW
  Temp: 44
  Evaporation: 7.03
  Moisture Start: 55.07
  Moisture End: 56.03
Sprklr3 reached